In [33]:
import torch

print('1. GPU 사용가능 -> True, GPU 사용불가 -> Flase')
print(torch.cuda.is_available())

1. GPU 사용가능 -> True, GPU 사용불가 -> Flase
True


In [34]:
USE_CUDA = torch.cuda.is_available()
print('2. GPU 사용중 -> True, GPU 사용안함 -> Flase')
print(USE_CUDA)

2. GPU 사용중 -> True, GPU 사용안함 -> Flase
True


In [35]:
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('3. 학습을 진행하는 기기:',device)

3. 학습을 진행하는 기기: cuda:0


In [36]:
print('4. GPU 이름 체크(cuda:0 에 연결된 그래픽 카드 기준)')
print(torch.cuda.get_device_name()) # 'NVIDIA TITAN X (Pascal)'

4. GPU 이름 체크(cuda:0 에 연결된 그래픽 카드 기준)
NVIDIA GeForce RTX 3050 Laptop GPU


In [37]:
print('5. 사용 가능 GPU 개수 체크')
print(torch.cuda.device_count()) # 3

5. 사용 가능 GPU 개수 체크
1


In [38]:
import torch
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())
tf.config.list_physical_devices('GPU')

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:',device)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17526994417015622135
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1751030171
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2180419307828140891
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
True
학습을 진행하는 기기: cuda:0


In [39]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [40]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [41]:
#GPU 사용
device = torch.device("cuda:0")

In [42]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. c:\Users\jinhee\project_readvice\readvice_KoBERT\kobert\.cache\kobert_v1.zip
using cached model. c:\Users\jinhee\project_readvice\readvice_KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [43]:
import pandas as pd
chatbot_data = pd.read_excel('../kobert/data/한국어_단발성_대화_데이터셋.xlsx')

In [44]:
chatbot_data.sample(n=10)


,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
18105,맨날오빠생각만해요,슬픔,NaN,NaN,NaN,NaN,NaN
35436,지코랑 설현이랑 헤어지게해주세요,혐오,NaN,NaN,NaN,NaN,NaN
15876,반드시처벌받게합시다두년들,분노,NaN,NaN,NaN,NaN,NaN
11825,역시 여자가 나서니 나라가 망해가네,분노,NaN,NaN,NaN,NaN,NaN
37409,이게 다 대통령 잘못 뽑아서 그래,혐오,NaN,NaN,NaN,NaN,NaN
3252,한번 할때마다 왼쪽으로 휘고있는걸 느껴요ㄱ,공포,NaN,NaN,NaN,NaN,NaN
21654,살도 엄.청 쪄서 뭘입어도 폼이안남 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ,슬픔,NaN,NaN,NaN,NaN,NaN
16113,한국에 이슬람사원 지으면 누군가가 불질러야 하고 대한민국 소방대원은 2시간후 출동 ...,분노,NaN,NaN,NaN,NaN,NaN
34154,"국민들 분열 시키지 말고, 민주주의 나라에서 투표로 결정해라... 일주일 지나 달라...",혐오,NaN,NaN,NaN,NaN,NaN
35458,친일 사학자들이 판치는 대한민국 현실,혐오,NaN,NaN,NaN,NaN,NaN


In [45]:
chatbot_data.loc[(chatbot_data['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [46]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고?', '1']
['당연히 그렇게 해야지 우리나라도 판매를 중단하라', '2']
['그거들은 뒤부터 미치겠어요...', '3']
['최악의 상황중 그나마 나은 방법이네. 기분은 잡치겠지만', '4']
['  요리하는것이 숙제하는것처럼 힘든저에게 용기나게 해주시고 할수 있을것같은 희망을 주셔서감사합니다!!', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


In [47]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [48]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [49]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [50]:
# Setting parameters
max_len = 64
batch_size = 5
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [51]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. c:\Users\jinhee\project_readvice\readvice_KoBERT\kobert\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [52]:
data_train[0]

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1]),
 array(42),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 5)

In [53]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)

In [54]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [56]:
import torch, gc
torch.cuda.empty_cache()

gc.collect()

595

In [57]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

c:\ProgramData\Anaconda3\envs\readvice\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [58]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

C:\Users\jinhee\AppData\Local\Temp\ipykernel_18132\2673822008.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/5789 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.2211496829986572 train acc 0.0
epoch 1 batch id 201 loss 1.906883955001831 train acc 0.1771144278606964
epoch 1 batch id 401 loss 1.9127635955810547 train acc 0.2054862842892774
epoch 1 batch id 601 loss 1.6356290578842163 train acc 0.2529118136439273
epoch 1 batch id 801 loss 1.9122178554534912 train acc 0.27565543071161025
epoch 1 batch id 1001 loss 1.8329330682754517 train acc 0.29810189810189736
epoch 1 batch id 1201 loss 1.6686519384384155 train acc 0.3095753538717717
epoch 1 batch id 1401 loss 2.4543402194976807 train acc 0.3189150606709472
epoch 1 batch id 1601 loss 2.3405873775482178 train acc 0.3266708307307907
epoch 1 batch id 1801 loss 1.5508021116256714 train acc 0.33714602998334103
epoch 1 batch id 2001 loss 1.6600208282470703 train acc 0.3452273863068457
epoch 1 batch id 2201 loss 1.2950736284255981 train acc 0.349750113584734
epoch 1 batch id 2401 loss 1.3879727125167847 train acc 0.35760099958350705
epoch 1 batch id 2601 loss 1.1453475952148438

C:\Users\jinhee\AppData\Local\Temp\ipykernel_18132\2673822008.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1930 [00:00<?, ?it/s]

epoch 1 test acc 0.514507772020726


  0%|          | 0/5789 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 1.1025818586349487 train acc 0.6
epoch 2 batch id 201 loss 1.7564709186553955 train acc 0.518407960199005
epoch 2 batch id 401 loss 1.7622041702270508 train acc 0.5147132169576063
epoch 2 batch id 601 loss 0.9127349853515625 train acc 0.5194675540765394
epoch 2 batch id 801 loss 1.568588376045227 train acc 0.5238451935081156
epoch 2 batch id 1001 loss 1.376814842224121 train acc 0.5270729270729277
epoch 2 batch id 1201 loss 1.632665991783142 train acc 0.5292256452955871
epoch 2 batch id 1401 loss 1.9804471731185913 train acc 0.5266238401142042
epoch 2 batch id 1601 loss 1.3607540130615234 train acc 0.5324172392254832
epoch 2 batch id 1801 loss 1.2397572994232178 train acc 0.5360355358134357
epoch 2 batch id 2001 loss 0.965359091758728 train acc 0.5437281359320316
epoch 2 batch id 2201 loss 0.974568247795105 train acc 0.5440254429804601
epoch 2 batch id 2401 loss 0.6465607285499573 train acc 0.5501041232819617
epoch 2 batch id 2601 loss 0.2950831651687622 train a

  0%|          | 0/1930 [00:00<?, ?it/s]

epoch 2 test acc 0.5471502590673564
